# 示例：发现毒蘑菇的相似特征

## 数据分析

为了找到毒蘑菇中存在的公共特征，可以运行Apriori算法来寻找包含特征值为2的频繁项集。

In [1]:
# 辅助函数
# 加载数据集
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

# 构建集合C1，大小为1的所有候选项集的集合。
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    # 对C1中每个项构建一个不变集合,
    # frozenset是冻结的集合，它是不可变的，存在哈希值，
    # 好处是它可以作为字典的key，也可以作为其它集合的元素。缺点是一旦创建便不能更改，没有add，remove方法。
    return list(map(frozenset, C1))   

# D：数据集，Ck：候选项集列表, minSupport：最小支持度
# 用于从C1生成L1，返回一个包含支持度值的字典
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                # 判断C1中的集合是记录的一部分
                if can not in ssCnt: ssCnt[can]=1
                # 增加字典中对应的计数值
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        # 计算支持度
        support = ssCnt[key]/numItems
        if support >= minSupport:
            #满足最小支持度要求，在列表的首部插入任意新的集合
            retList.insert(0,key)
        # 返回最频繁项集的支持度
        supportData[key] = support
    return retList, supportData

In [2]:
# Apriori算法
# Lk：频繁项集列表, k：项集元素个数
# 输出Ck
def aprioriGen(Lk, k): 
    # 创建一个空列表
    retList = []
    # 计算Lk中的元素数目
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            # 取两个集合前面k-2个元素
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            # 进行排序
            L1.sort(); L2.sort()
            if L1==L2: 
                # 如果前面k-2个元素都相等，那么就将这两个集合合成一个大小为k的集合
                retList.append(Lk[i] | Lk[j])
    return retList

# dataSet：数据集，minSupport：最小支持度
def apriori(dataSet, minSupport = 0.5):
    # 创建C1
    C1 = createC1(dataSet)
    # 读入数据集将其转化为D（集合列表）
    D = list(map(set, dataSet))
    # 创建L1频繁项集
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    # 设定项集元素
    k = 2
    while (len(L[k-2]) > 0):
        # 得到Ck，创建包含更大项集的更大列表
        Ck = aprioriGen(L[k-2], k)
        # 得到Lk, 丢掉不满足最小支持度要求的项集
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        # Lk列表被添加到L
        L.append(Lk)
        # 增加k的值，
        k += 1
    return L, supportData

In [3]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]

In [4]:
L, suppData = apriori(mushDatSet, minSupport=0.3)

In [5]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'34', '2'})
frozenset({'2', '36'})
frozenset({'59', '2'})
frozenset({'2', '63'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'85', '2'})
frozenset({'2', '86'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'2', '39'})


In [6]:
for item in L[3]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '34', '2', '63'})
frozenset({'28', '34', '2', '85'})
frozenset({'28', '34', '2', '86'})
frozenset({'28', '34', '2', '90'})
frozenset({'28', '34', '2', '39'})
frozenset({'28', '34', '2', '53'})
frozenset({'59', '28', '2', '39'})
frozenset({'59', '28', '34', '2'})
frozenset({'59', '28', '2', '63'})
frozenset({'59', '28', '85', '2'})
frozenset({'59', '28', '2', '86'})
frozenset({'59', '28', '2', '90'})
frozenset({'28', '85', '2', '63'})
frozenset({'28', '85', '2', '86'})
frozenset({'28', '85', '2', '39'})
frozenset({'28', '85', '2', '53'})
frozenset({'28', '2', '63', '86'})
frozenset({'28', '2', '85', '90'})
frozenset({'28', '2', '90', '86'})
frozenset({'28', '2', '39', '90'})
frozenset({'28', '2', '53', '90'})
frozenset({'28', '2', '39', '63'})
frozenset({'28', '2', '39', '86'})
frozenset({'28', '2', '53', '86'})
frozenset({'28', '2', '53', '39'})
frozenset({'34', '2', '53', '39'})
frozenset({'34', '2', '85', '53'})
frozenset({'34', '2', '53', '86'})
frozenset({'34', '2'